In [1]:
!pip install scikit-multilearn

In [3]:
# Basic
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

# Pre-processing
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import IterativeStratification

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.problem_transform import LabelPowerset
from sklearn.ensemble import RandomForestClassifier
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.adapt import MLkNN
from skmultilearn.ensemble import LabelSpacePartitioningClassifier

# Optimization
from sklearn.model_selection import GridSearchCV

# Evaluation
from sklearn.metrics import hamming_loss
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_score

In [9]:
# Loading X data
train_features = pd.read_csv("./data/train_features.csv")
test_features = pd.read_csv("./data/test_features.csv")

In [10]:
# Loading Y data
train_targets_sc = pd.read_csv("./data/train_targets_scored.csv")
train_targets_ns = pd.read_csv("./data/train_targets_nonscored.csv")

In [11]:
sample_submission = pd.read_csv("./data/sample_submission.csv")

In [12]:
X = train_features.drop('sig_id', axis = 1)
X = pd.get_dummies(X)

In [13]:
y = train_targets_sc.drop('sig_id', axis = 1)

In [14]:
stratifier = IterativeStratification(n_splits=2, order=2, sample_distribution_per_fold=[0.25, 0.75])
train_indexes, test_indexes = next(stratifier.split(X, y))

X_train, y_train = X.loc[train_indexes], y.loc[train_indexes]
X_val, y_val = X.loc[test_indexes], y.loc[test_indexes]

In [36]:
knn = MLkNN(k = 5)

In [37]:
%%time
knn.fit(X_train.values, y_train.values)

/home/andrefonseca/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_neighbors=5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


CPU times: user 7min 43s, sys: 559 ms, total: 7min 44s
Wall time: 7min 44s


MLkNN(k=5)

In [38]:
y_pred = knn.predict(X_val)

In [40]:
y_prob = knn.predict_proba(X_val)

In [41]:
y_prob_array = y_prob.toarray()

In [42]:
logging_score = log_loss(y_val.values, y_prob_array)

In [43]:
print(f'Log-loss score: {logging_score}')

Log-loss score: 3.518411840190982


In [44]:
with open('./data/models/mlknn_model.pkl','wb') as outfile:
  pickle.dump(knn, outfile)

In [48]:
clusterer = pickle.load(open('./data/clusterer.pkl', 'rb'))

In [49]:
lab_space = LabelSpacePartitioningClassifier(
    classifier = BinaryRelevance(
    classifier = RandomForestClassifier(),
    require_dense = [False, True]
    ),
    clusterer  = clusterer
)

In [50]:
%%time
lab_space.fit(X_train,y_train)

CPU times: user 2h 44min 43s, sys: 16.5 s, total: 2h 44min 59s
Wall time: 2h 45min 51s


LabelSpacePartitioningClassifier(classifier=BinaryRelevance(classifier=RandomForestClassifier(),
                                                            require_dense=[False,
                                                                           True]),
                                 clusterer=<skmultilearn.cluster.networkx.NetworkXLabelGraphClusterer object at 0x7ff1421b9320>,
                                 require_dense=[False, False])

In [52]:
y_pred = lab_space.predict(X_val)

In [53]:
y_prob = lab_space.predict_proba(X_val)

In [54]:
y_prob_array = y_prob.toarray()

In [55]:
accuracy_score = accuracy_score(y_val.values, y_pred)

In [56]:
print(f'Accuracy score: {accuracy_score}')

Accuracy score: 0.4559959690964058


In [57]:
f1_score = f1_score(y_val.values, y_pred, average = 'micro')

In [58]:
print(f'F1-score: {accuracy_score}')

F1-score: 0.4559959690964058


In [59]:
logging_score = log_loss(y_val.values, y_prob_array)

In [60]:
print(f'Log-loss score: {logging_score}')

Log-loss score: 10.307731452603159


In [51]:
with open('./data/models/lab_space.pkl','wb') as outfile:
  pickle.dump(lab_space, outfile)

In [61]:
br = pickle.load(open('./data/models/binary_forest_model.pkl', 'rb'))

In [62]:
y_pred = br.predict(X_val)

In [63]:
y_prob = br.predict_proba(X_val)

In [64]:
y_prob_array = y_prob.toarray()

In [65]:
logging_score = log_loss(y_val.values, y_prob_array)

TypeError: 'numpy.float64' object is not callable

In [66]:
print(f'Log-loss score: {logging_score}')

Log-loss score: 10.307731452603159
